In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

In [4]:
load_dotenv()
GROQ_API_KEY =os.environ.get("GROQ_API_KEY")


In [5]:
LLM=ChatGroq(model="llama-3.3-70b-versatile")

# Runnable Sequence

In [25]:
prompt1 =PromptTemplate(template="write a joke with emojis about {topic} ",
                       input_variables=['topic'])

prompt2 = PromptTemplate(template="Explain the following a joke - {text}",
                         input_variables=['text'])
parser=StrOutputParser()
model=ChatGroq(model="llama-3.3-70b-versatile",temperature=1)

#chain = RunnableSequence(prompt1,model,parser,prompt2,model,parser)

#In LCEL
chain =prompt1 |model|parser|prompt2|model|parser

res=chain.invoke({"topic":"human"})

In [26]:
print(res)

A clever joke. Let's break it down:

The joke starts by setting up a scenario: a human (represented by the 🏃‍♂️ emoji) brings a ladder (🚪) to a party (🎉). This is an unusual and unexpected thing to do, which piques the listener's interest.

The punchline is: "Because they wanted to take their social life to the next level." Here, the phrase "to the next level" has a double meaning:

1. **Literal meaning**: In a physical sense, a ladder allows you to climb up to a higher level or elevation.
2. **Figurative meaning**: In a social context, "taking something to the next level" means to improve or elevate it, often implying an increase in status, quality, or success. In this case, it means to enhance one's social life or relationships.

The humor comes from the wordplay between the literal and figurative meanings of "to the next level." The joke relies on the listener being familiar with the idiomatic expression and making the mental connection between the physical ladder and the social asp

# RunnableParallel

In [8]:
# use the two llm (tweet and linkedln)
from langchain.schema.runnable import RunnableParallel


In [9]:
prompt1 =PromptTemplate(template="Generate a tweet about {topic}",
                        input_variables=["topic"])

prompt2 = PromptTemplate(template="Generate a Linkedin post about {topic}",
                         input_variables=["topic"]
                         )

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    "tweet":RunnableSequence(prompt1,model,parser),
    "linkedin":RunnableSequence(prompt2,model,parser)
})

result =parallel_chain.invoke({"topic":"AI"})


In [10]:
print(result)

{'tweet': '"Artificial Intelligence is revolutionizing the world. From personalized recommendations to life-saving medical diagnoses, AI is transforming the way we live and work. What\'s the most exciting AI innovation you\'ve seen lately? #AI #MachineLearning #FutureTech"', 'linkedin': "Here's a potential LinkedIn post about AI:\n\n**The Future of Work: Embracing the Power of AI**\n\nAs we continue to navigate the ever-changing landscape of technology, one thing is clear: Artificial Intelligence (AI) is revolutionizing the way we work and live.\n\nFrom automating routine tasks to analyzing complex data sets, AI is opening up new opportunities for innovation, efficiency, and growth. But what does this mean for us as professionals?\n\nFor one, it means that we need to be adaptable and willing to learn new skills. As AI takes over repetitive tasks, we'll be free to focus on higher-level thinking, creativity, and problem-solving. It's an exciting time to be in the workforce, with new poss

In [11]:
parallel_chain.invoke({"topic":"Generative AI which I start "})

{'tweet': '"Starting to explore the incredible world of Generative AI and I\'m blown away by its potential to revolutionize art, music, and writing What are some of the most exciting applications you\'ve seen so far? #GenerativeAI #AIart #FutureOfCreativity"',
 'linkedin': 'Here\'s a potential LinkedIn post about Generative AI:\n\n**"The Future is Here: Unlocking the Power of Generative AI"**\n\nI\'m excited to start a conversation about one of the most transformative technologies of our time: Generative AI. This innovative field has the potential to revolutionize industries, from art and design to healthcare and beyond.\n\nGenerative AI models, such as Generative Adversarial Networks (GANs) and Variational Autoencoders (VAEs), have the ability to generate new, synthetic data that\'s often indistinguishable from the real thing. This technology is already being used to create stunning works of art, generate realistic synthetic data for training AI models, and even develop new medicines.

# RunnablePassthrough  
**It is a special runnable primitive that simply returns the input as output without modifying it**

In [12]:
from langchain.schema.runnable import RunnablePassthrough,RunnableParallel,RunnableSequence

In [13]:
prompt1 =PromptTemplate(template="write a joke with about {topic}",
                       input_variables=['topic'])

prompt2 = PromptTemplate(template="Explain the following a joke - {text}",
                         input_variables=['text'])

joke_gen_chain = RunnableSequence(prompt1,model,parser)

parallel_chain = RunnableParallel({
    "joke":RunnablePassthrough(),
    "Explainable":RunnableSequence(prompt2,model,parser)

})

fianl_chain =RunnableSequence(joke_gen_chain,parallel_chain)

res=fianl_chain.invoke({"topic":"cricket"})


In [16]:
print(res['joke'])

Why did the cricket go to the doctor?

Because it had a bowled-over stomach and was feeling a little "pitch"-sick.


In [17]:
print(res['Explainable'])

A joke that's a perfect pitch (pun intended) for cricket enthusiasts. Let's break it down:

The joke is a play on words, using cricket terminology to create a pun. Here's how it works:

* "Bowled-over" is a phrase that can mean being completely surprised or overwhelmed, but in cricket, a bowler is a player who delivers the ball to the batsman. So, "bowled-over stomach" is a clever wordplay on the phrase, implying that the cricket's stomach is upset, much like a batsman might be bowled over (i.e., knocked over) by a fast ball.
* "Pitch-sick" is another clever play on words. In cricket, the pitch refers to the rectangular area in the center of the field where the ball is bowled. However, "pitch" sounds similar to "pitch" as in "feeling sick" (nauseous). So, the joke is saying that the cricket is feeling a little unwell, like it has a stomach bug, and the word "pitch" is used to create a pun on the cricket term.

The joke relies on a basic understanding of cricket terminology, but the wor

# Runnable Lambda  
**It is allows you to apply custom python functions within an AI pipeline.**

In [ ]:
from langchain.schema.runnable import RunnableLambda
def word_count(text):
    return len(text.split())

joke_gen_chain = RunnableSequence(prompt1,model,parser)

parallel_chain = RunnableParallel({
    "joke":RunnablePassthrough(),
    # "word_count":RunnableLambda(word_count)
    "word_count":RunnableLambda(lambda x:len(x.split()))
})

final_chains = RunnableSequence(joke_gen_chain,parallel_chain)

final_chains.invoke({"topic":"AI"})


{'joke': 'Why did the AI program go to therapy?\n\nBecause it was struggling to process its emotions.',
 'word_count': 16}

# RunnableBranch  
**it is used when have the conditions. It work as the if else**

In [19]:
from langchain.schema.runnable import RunnableBranch

In [ ]:


prompt = PromptTemplate(template="Write a detailed report on {topic}",
                        input_variables=["topic"])


prompt2 = PromptTemplate(template="Summarize the following text \n {text}",
                         input_variables=['text'])

report_gen = RunnableSequence(prompt,model,parser)

#Create the chain if report is the > 500 words then summarize it.
branch_chain = RunnableBranch(
    #X is the output of report
    (lambda x:len(x.split()) > 500,RunnableSequence(prompt2,model,parser)),# if condition
    RunnablePassthrough() # Otherwise return the same output # else condition
)

chain = RunnableSequence(report_gen,branch_chain)

In [21]:
report = chain.invoke({"topic":"Openai vs deepseek"})

In [22]:
print(report)

The text provides a detailed comparison between OpenAI and DeepSeek, two prominent players in the natural language processing (NLP) space. Here's a summary of the key points:

**Overview of OpenAI and DeepSeek:**

* OpenAI is a non-profit research organization founded in 2015 by Elon Musk and others, with a focus on developing AI that benefits humanity.
* DeepSeek is a lesser-known but rapidly growing player in the NLP space, founded in 2018, with a focus on developing efficient and scalable language models.

**Key Features and Models:**

* OpenAI: Developed models such as GPT-3, GPT-2, and DALL-E, which can generate human-like text, answer questions, and create images from text prompts.
* DeepSeek: Developed models with efficient architecture, domain-specific models, and explainability features, making them suitable for industrial applications.

**Comparison:**

* Model architecture: OpenAI uses transformer-based models, while DeepSeek uses a combination of convolutional and recurrent